# Time-Dependent disk

The evolution equation for the disk profile $\Sigma(r,t)$ is:
$$\frac{\partial \Sigma}{\partial t} = \frac{3}{r}\frac{\partial}{\partial r}\left(r^{1/2}\frac{\partial}{\partial r} [\nu \Sigma r^{1/2}]\right)$$

Here, we have assumed that $\Omega =\Omega_k= \sqrt{GM/r^3$

## Thin disk approximation:

- $$\rho = \frac{\Sigma}{H}$$
- $$ H = R\frac{c_s}{v_\phi}$$

with $c_s^2=p/\rho$ is the sound speed. With the Shakura \& Sunyaev description for the viscosity ($\nu=\alpha c_s H$):
$$ c_s^2 = \frac{\nu}{\alpha}\Omega$$

For a radiative energy transport:
$$F=\frac{4\sigma_{SB}}{3 \kappa \rho}\frac{T_c^4}{H}$$

should equals the viscous heat:

\begin{equation}
\frac{4\sigma_{SB}}{3\tau}T_c^4=D(R)=\frac{9}{8}\nu\Sigma \Omega^2
\end{equation}

Asuming that the opacity has the form $\kappa=\kappa_o \rho^aT^b$, the optical depht can be written as:

$$ \tau = \kappa\Sigma = \kappa_o\frac{\Sigma^{a+1}T^b}{H^a} \rightarrow \nu = \frac{32 \sigma_{SB}}{27} \frac{T^{4-b}}{\kappa_0\Sigma^{a+1}\Omega^2}\left(\frac{\nu}{\alpha c_s}\right)^{a}$$

Also, assuming an ideal gass law and gas preassure dominates over radiation pressure: 
$$ T = \frac{P}{\rho}\frac{\mu m_p}{\kappa_B}=c_s^2 \frac{\mu m_p}{\kappa_B}  $$

Then



$$  \nu^{1-a} = \frac{32 \sigma_{SB}}{27} \frac{c_s^{2(4-b)-a}}{\kappa_0\Sigma^{a+1}\Omega^2}\left(\frac{\mu m_p}{\kappa_B}  \right)^{4-b}\left(\frac{1}{\alpha }\right)^{a} = \frac{32 }{27} \frac{\sigma_{SB}}{\kappa_0\Sigma^{a+1}\Omega^2}\left(\frac{\mu m_p}{\kappa_B}  \right)^{4-b}\left(\frac{1}{\alpha }\right)^{a}\left(\frac{\nu}{\alpha}\Omega\right)^{(2(4-b)-a)/2} $$

Then, the viscosity can be write as:
$$\nu = C r^p \Sigma^q$$

with 
- $$p=3\left(\frac{1}{2}-\frac{a+1}{6-2b+a}\right)$$
- $$q = \frac{2(a+2)}{6-2b+a}$$
- $$C=\alpha^{1+\frac{2}{6-2b+a}}\left(\frac{27\kappa_0}{32\sigma_{SB}}\right)^{\frac{2}{6-2b+a}}\left(\frac{\kappa_B}{\mu m_p}\right)^{1+\frac{2-a}{6-2b+a}}\left(GM\right)^{\frac{a+1}{6-2b+a}-\frac{1}{2}}$$

Making:
-$$\sigma=\frac{\Sigma}{\Sigma_o}\qquad R=\frac{r}{r_0} \; \mathrm{and} \qquad \tau=\frac{t}{t_0}$$

The first equation is:
The evolution equation for the disk profile $\Sigma(r,t)$ is:
$$\frac{\partial \sigma}{\partial \tau} = \frac{3}{R}\frac{\partial}{\partial R}\left(R^{1/2}\frac{\partial}{\partial R} [\hat{\nu} \sigma R^{1/2}]\right)$$

with $t_0=3r_0^2/4\nu_0$.

Additionally making $x=\sqrt{r}$ and $\sigma= SR^{-3/2}$

$$\frac{\partial S}{\partial \tau} = \frac{\partial^2}{\partial x^2} \left( \frac{S\hat{\nu}}{x^2}\right)$$

In general: $\hat{\nu}=x^{2p-3q}S^{q}$
